利用vgg提取每一类的平均的特征，保存特征

In [1]:
import os
import glob
import json
import torch
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image
#from progressbar import * #进度条
from tqdm import tqdm #进度条
from torchnet import meter
from natsort import natsorted
import matplotlib.pyplot as plt
import torch.nn.functional as F #torch是关于运算的包
from torch.autograd import Variable
from torch.utils.data import Dataset
from torchvision import transforms as T
from torchvision import datasets,transforms, models #torchvision则是打包了一些数据集
os.environ['CUDA_VISIBLE_DEVICES']='2'

### Vgg模型 

In [140]:
train_csv = pd.read_csv("data/train.csv")
#temp_df = train_csv.groupby('Id')['Id'].agg({'cnt':'count'})
temp_df = train_csv['Id'].value_counts()
print(temp_df)
#temp_df_subset = temp_df[temp_df.Image >= 30]
#print(temp_df_subset)

new_whale    9664
w_23a388d      73
w_9b5109b      65
w_9c506f6      62
w_0369a5c      61
w_700ebb4      57
w_3de579a      54
w_564a34b      51
w_fd3e556      50
w_88e4537      49
w_2b069ba      48
w_d405854      47
w_f0fe284      45
w_789c969      45
w_343f088      40
w_5e8e218      40
w_778e474      40
w_60ce6fc      37
w_5a2634c      37
w_a9304b9      37
w_6822dbc      36
w_af367c3      35
w_f765256      34
w_1ca9ab1      34
w_17b0d3a      33
w_d72771c      32
w_6cda039      31
w_8c25681      31
w_08630fd      31
w_04003e9      30
             ... 
w_3791ec3       1
w_ca93339       1
w_7d02551       1
w_bc4da71       1
w_7c26ef7       1
w_c1bab16       1
w_a42c505       1
w_6441130       1
w_2980f85       1
w_297e53a       1
w_5fcd8d4       1
w_e014c37       1
w_7efe670       1
w_17be6f2       1
w_c5a7b38       1
w_bbdfec9       1
w_90df70c       1
w_58ddf90       1
w_391ef09       1
w_210b625       1
w_0a1fcd6       1
w_ecad2c9       1
w_0af3867       1
w_dc4c211       1
w_a0c732f 

In [3]:
#vgg_model = models.vgg16(pretrained=True)
#print(vgg_model)
class Encoder(nn.Module):
    '''
    提取vgg的4096向量
    '''
    def __init__(self):
        super(Encoder, self).__init__()
        VGG = models.vgg16(pretrained=True)
        self.feature = VGG.features
        self.classifier = nn.Sequential(*list(VGG.classifier.children())[:-3])
        pretrained_dict = VGG.state_dict()
        model_dict = self.classifier.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.classifier.load_state_dict(model_dict)
    def forward(self, x):
        output = self.feature(x)
        output = output.view(output.size(0), -1)
        output = self.classifier(output)
        return output

In [4]:
#print(Encoder())
def extractor(img_path, net):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])
    img = Image.open(img_path).convert("RGB")
    img = transform(img)
    img = Variable(torch.unsqueeze(img, dim=0).float(), requires_grad=False)#增加一维
    
    net = net.cuda()
    img = img.cuda()
    y = net(img).cpu()
    y = torch.squeeze(y)
    y = y.data.numpy()
    
    return y

In [5]:
img_path = 'data/train/1cf255a78.jpg'
feature = extractor(img_path, Encoder())
print(feature.shape)

(4096,)


### 提取特征

In [115]:
def extract_train_feature(dir_path, model, feature_dim=4096):
    data = np.zeros((1, feature_dim))
    label = []
    for file in tqdm(os.listdir(dir_path)):
        if file == 'new_whale':
            continue
        path = os.path.join(dir_path, file)
        if os.path.isdir(path):
            feature_tmp = np.zeros((1, feature_dim))
            label_name = file
            img_num = 0
            for img in os.listdir(path):
                img_path = os.path.join(path, img)
                feature = extractor(img_path, model)
                img_num += 1
                feature_tmp += feature
            if img_num > 0:
                feature = feature_tmp / img_num
                data = np.concatenate((data, feature))
                label.append(label_name)
    data = data[1:, :]
    np.savetxt('model/train_feature.txt', data, fmt='%f')
    label_file = open('model/label.txt', 'w')
    json.dump(label, label_file)
    label_file.close

In [23]:
extract_train_feature('data/train_classes', Encoder())

100%|██████████| 5005/5005 [10:12<00:00,  8.17it/s]


### 找到最近的5个类别

In [39]:
train_features = np.loadtxt('model/train_feature.txt', dtype=float)
train_labels = open('model/label.txt', 'r')
labels = json.load(train_labels)
train_labels.close()
#print(train_features)
#print(labels)

In [116]:
def top_5_classes(test_feature, train_features, labels):
    #print(train_features.shape)
    distance = train_features - test_feature
    euclidean_distance = np.linalg.norm(distance, axis=1, keepdims=True)
    euclidean_distance = euclidean_distance.flatten()
    min_distance_idx = np.argsort(euclidean_distance)[:5]
    min_distance = np.sort(euclidean_distance)[:5]
    #print(min_distance)
    classes = []
    for idx in min_distance_idx:
        if euclidean_distance[idx] > 40:
            classes.append('new_whale')
        else:
            classes.append(labels[idx])
    return classes

In [117]:
test_img = 'data/train/8c777dd58.jpg'
feature = extractor(test_img, Encoder())
predict_label = top_5_classes(feature, train_features, labels)
print(predict_label)

['new_whale', 'new_whale', 'new_whale', 'new_whale', 'new_whale']


In [118]:
def submmit(test_floder, model, output_file):
    files = natsorted(glob.glob(test_floder + '/*'))
    f = open(output_file, 'w')
    f.write('Image,Id\n')
    for file in tqdm(files):
        feature = extractor(file, model)
        top_5 = top_5_classes(feature, train_features, labels)
        img_name = file.split('/')[-1]
        f.write(' '.join([img_name+',', top_5[0], top_5[1], top_5[2], top_5[3],top_5[4], '\n']))
    f.close()

In [119]:
submmit('data/test', Encoder(), 'submission.csv')

100%|██████████| 7960/7960 [31:26<00:00,  4.06it/s]
